# Annotation Generation Tools

In [ ]:
import json
import os
from collections import OrderedDict
import fnmatch
import re

In [ ]:
output_dir = '../../Annotations'
temp_output_dir = './temp'



task = "classification" # "classification" or "segmentation" or "cpr_quality"
task = "segmentation" # "classification" or "segmentation" or "cpr_quality"
task = "cpr_quality" # "classification" or "segmentation" or "cpr_quality"

# root_dir = '/standard/UVA-DSA/NIST EMS Project Data/EgoExoEMS_CVPR2025/Dataset/Final'  # Replace with your directory path
# root_dir = '/standard/UVA-DSA/NIST EMS Project Data/DataCollection_Spring_2025/CARS/organized'  # Replace with your directory path
root_dir = '/standard/UVA-DSA/NIST EMS Project Data/EgoEMS_AAAI2026/'  # Replace with your directory path

# output_file = f'{temp_output_dir}/main_annotation.json' # temp
output_file = f'{output_dir}/aaai26_main_annotation_{task}.json'
# output_file = f'{output_dir}/main_annotation_{task}.json'

# remove existing output file if it exists
if os.path.exists(output_file):
    os.remove(output_file)


# Production Ready Version

In [ ]:
# production ready
is_production = True

In [ ]:
import os
import fnmatch
from collections import OrderedDict

accepted_stream_types = [
    'ego', 'clip_ego', 'resnet_ego',
    'distance_sensor_data',
    'smartwatch_data', 'audio', 'i3d_flow', 'i3d_rgb', 'resnet_ego',
    'BBOX_MASKS'
]

# --- UPDATED: no remapping; directory name is the stream type ---
def get_stream_type(directory_name: str) -> str:
    """Return the canonical stream type from the directory name (no aliases)."""
    return directory_name

# --- UPDATED: parsing rules aligned to accepted_stream_types ---
def parse_file(file_path: str, stream_type: str):
    """
    Decide whether to keep this file for the given stream_type.
    NOTE: For 'audio', JSONs are handled outside this function and placed
    into 'audio_transcript' (virtual stream key).
    """
    file_name = os.path.basename(file_path)
    file_id, ext = os.path.splitext(file_name)
    ext = ext.lower()

    if stream_type == 'ego':
        # Keep .mp4 that end with *_rgb_final or *_rgb_partial AND any .json (e.g., metadata)
        if ext == '.mp4':
            if not (file_id.endswith("rgb_final") or file_id.endswith("rgb_partial")):
                return None
        elif ext == '.json' and file_id.endswith("annotation"):
            pass
        else:
            return None

    elif stream_type == 'audio':
        # Only audio binaries here (.mp3/.wav). JSON is routed to 'audio_transcript' outside.
        if ext not in ('.mp3', '.wav'):
            return None

    elif stream_type == 'distance_sensor_data':
        if ext != '.csv':
            return None

    elif stream_type == 'smartwatch_data':
        # Keep only synced CSVs
        if ext != '.csv':
            return None

    elif stream_type in ('i3d_flow', 'i3d_rgb'):
        if ext != '.npy':
            return None
        if stream_type == 'i3d_flow' and 'flow' not in file_id:
            return None
        if stream_type == 'i3d_rgb'  and 'rgb'  not in file_id:
            return None

    elif stream_type in ('resnet_ego', 'clip_ego'):
        if ext != '.npy':
            return None

    elif stream_type == 'BBOX_MASKS':
        # Allow common mask formats; adjust if you have a stricter spec
        if ext not in ('.png', '.npy', '.json'):
            return None

    else:
        # Any other stream types are not supported
        return None

    return {
        "file_id":   file_id,
        "file_path": file_path,
    }

def normalize_subject_key(subject_key: str) -> str:
    return subject_key

# --- UPDATED: only strip .txt globally; do not remove images anymore ---
def process_files(files):
    return [f for f in files if not f.endswith('.txt')]


# ---------- helpers for ego “Verified” selection ----------
_VERIFIED_PREFIX_RE = re.compile(r'^(?:Verified[\s_-]*)', re.IGNORECASE)

def _strip_verified_prefix(s: str) -> str:
    return _VERIFIED_PREFIX_RE.sub('', s)

def _is_verified_name(s: str) -> bool:
    return bool(_VERIFIED_PREFIX_RE.match(s))

def process_directory(root_path):
    subjects = []
    for root, dirs, files in os.walk(root_path):
        parts = root.split(os.sep)
        # Keep your depth constraint if it reflects your dataset layout
        if len(parts) != 9:
            continue

        subj_key    = normalize_subject_key(parts[-4])
        scen_id     = parts[-3]
        trial_id    = parts[-2]
        stream_name = parts[-1]

        # Only accept directories that are in the canonical list
        if subj_key.startswith('ld') or stream_name not in accepted_stream_types:
            continue

        stream_type = get_stream_type(stream_name)
        files = process_files(files)
        if not files:
            continue

        # --- subject, scenario, trial setup ---
        subj = next((s for s in subjects if s['subject_id'] == subj_key), None)
        if not subj:
            subj = OrderedDict([
                ("subject_id", subj_key),
                ("expertise_level", "EMT"),
                ("scenarios", [])
            ])
            subjects.append(subj)

        scen = next((sc for sc in subj['scenarios'] if sc['scenario_id'] == scen_id), None)
        if not scen:
            scen = OrderedDict([("scenario_id", scen_id), ("trials", [])])
            subj['scenarios'].append(scen)

        trial = next((t for t in scen['trials'] if t['trial_id'] == trial_id), None)
        if not trial:
            trial = OrderedDict([
                ("trial_id", trial_id),
                ("streams", OrderedDict()),
                ("keysteps", OrderedDict()),
            ])
            scen['trials'].append(trial)

        streams = trial['streams']

        # --- UPDATED: split audio transcripts from audio binaries ---
        if stream_type == 'audio':
            for fname in sorted(files):
                full = os.path.join(root, fname)
                _, ext = os.path.splitext(fname)
                ext = ext.lower()

                if ext == '.json' and fname.endswith('_deidentified.json'):
                    # Route transcript JSON into a separate stream key
                    file_id, _ = os.path.splitext(os.path.basename(full))
                    info = {"file_id": file_id, "file_path": full}
                    streams.setdefault('audio_transcript', []).append(info)
                else:
                    info = parse_file(full, 'audio')
                    if info:
                        streams.setdefault('audio', []).append(info)

        
        # --- ego: prefer Verified* when duplicates exist ---
        elif stream_type == 'ego':
            # collect valid mp4s (by suffix rule) and jsons
            candidate_mp4s = []
            jsons = []
            for fname in sorted(files):
                full = os.path.join(root, fname)
                info = parse_file(full, 'ego')
                if not info:
                    continue
                if full.lower().endswith('.mp4'):
                    candidate_mp4s.append(info)
                elif full.lower().endswith('.json'):
                    jsons.append(info)

            # Deduplicate by normalized id (strip 'Verified' prefix),
            # preferring files whose base name starts with Verified*
            chosen_by_norm = {}
            for info in candidate_mp4s:
                file_id = info["file_id"]
                norm_id = _strip_verified_prefix(file_id)
                is_verified = _is_verified_name(file_id)
                prev = chosen_by_norm.get(norm_id)
                if prev is None or (is_verified and not _is_verified_name(prev["file_id"])):
                    chosen_by_norm[norm_id] = info

            # Save selected videos
            if chosen_by_norm:
                streams.setdefault('ego', []).extend(sorted(chosen_by_norm.values(), key=lambda x: x["file_id"]))
            # Save jsons as usual
            if jsons:
                streams.setdefault('ego', []).extend(sorted(jsons, key=lambda x: x["file_id"]))

        else:
            # Standard handling for all other streams
            for fname in sorted(files):
                full = os.path.join(root, fname)
                info = parse_file(full, stream_type)
                if info:
                    streams.setdefault(stream_type, []).append(info)

    # sort for consistency
    subjects.sort(key=lambda s: s['subject_id'])
    for subj in subjects:
        subj['scenarios'].sort(key=lambda sc: sc['scenario_id'])
        for scen in subj['scenarios']:
            scen['trials'].sort(key=lambda t: t['trial_id'])

    return subjects

def generate_json_structure(root_directory, version="v1.2025.07.07"):
    return OrderedDict([
        ("subjects", process_directory(root_directory)),
        ("version",  version)
    ])


In [ ]:
! pwd

In [ ]:

json_data = generate_json_structure(root_dir)

# sort the json structure
json_data = dict(sorted(json_data.items()))
with open(output_file, 'w') as json_file:
    json.dump(json_data, json_file, indent=4)

print(f"JSON structure saved to {output_file}")


## Sanity Check

In [ ]:
import json
from collections import defaultdict, OrderedDict
from typing import Dict, List, Tuple, Union, Iterable

TrialID = str

def _iter_trials(data: Dict) -> Iterable[Tuple[TrialID, Dict]]:
    """
    Yield (trial_key, trial_dict) across all subjects/scenarios/trials.
    trial_key format: "<subject_id>/<scenario_id>/<trial_id>"
    """
    subjects = data.get("subjects", [])
    for subj in subjects:
        sid = subj.get("subject_id", "UNKNOWN_SUBJECT")
        for scen in subj.get("scenarios", []):
            scid = scen.get("scenario_id", "UNKNOWN_SCENARIO")
            for trial in scen.get("trials", []):
                tid = trial.get("trial_id", "UNKNOWN_TRIAL")
                trial_key = f"{sid}/{scid}/{tid}"
                yield trial_key, trial

def _count_files_in_stream(stream_items) -> int:
    """Count files in a stream list (list of dicts with file_id/file_path)."""
    if not isinstance(stream_items, list):
        return 0
    return sum(1 for _ in stream_items)

def sanity_check_dataset(
    dataset: Union[str, Dict],
    required_streams: Iterable[str] = ("smartwatch_data",),
    also_show_empty_streams: bool = False,
) -> Dict:
    """
    Sanity check the generated dataset JSON.

    Args:
        dataset: dict already loaded OR path to a JSON file.
        required_streams: streams that must exist (>=1 file) in every trial.
        also_show_empty_streams: if True, list trials where a stream key exists but has 0 files.

    Returns:
        A dict with summary, per_stream stats, and missing required streams.
    """
    # Load if a path is provided
    if isinstance(dataset, str):
        with open(dataset, "r", encoding="utf-8") as f:
            data = json.load(f)
    else:
        data = dataset

    # Collect trials
    trial_list: List[Tuple[TrialID, Dict]] = list(_iter_trials(data))
    total_trials = len(trial_list)

    # Per-trial stream file counts
    per_trial_stream_counts: Dict[TrialID, Dict[str, int]] = {}
    # Per-stream coverage
    stream_total_files: Dict[str, int] = defaultdict(int)
    stream_trials_having_any: Dict[str, int] = defaultdict(int)
    stream_trials_with_zero: Dict[str, List[TrialID]] = defaultdict(list)

    # Track missing required streams per trial
    missing_required_by_stream: Dict[str, List[TrialID]] = {s: [] for s in required_streams}

    for trial_key, trial in trial_list:
        streams: Dict[str, list] = trial.get("streams", {}) or {}
        # Count files per stream for this trial
        this_trial_counts: Dict[str, int] = {}
        for stream_name, items in streams.items():
            n = _count_files_in_stream(items)
            this_trial_counts[stream_name] = n
            stream_total_files[stream_name] += n
            if n > 0:
                stream_trials_having_any[stream_name] += 1
            else:
                stream_trials_with_zero[stream_name].append(trial_key)

        # For streams that never appear as a key in this trial, they count as zero here:
        all_stream_names_seen = set(this_trial_counts.keys())
        # You may prefer to evaluate required streams even if they don't exist as keys.
        for req in required_streams:
            n = this_trial_counts.get(req, 0)
            if n <= 0:
                missing_required_by_stream.setdefault(req, []).append(trial_key)

        per_trial_stream_counts[trial_key] = this_trial_counts

    # Build per-stream report
    per_stream_report = OrderedDict()
    all_streams = sorted(set(stream_total_files.keys()) | set(stream_trials_with_zero.keys()))
    for s in all_streams:
        per_stream_report[s] = {
            "total_files": stream_total_files.get(s, 0),
            "trials_with_at_least_one": stream_trials_having_any.get(s, 0),
            "trials_missing_or_zero": (
                stream_trials_with_zero.get(s, [])  # keys present but zero files
            ),
        }

    result = {
        "summary": {
            "total_subjects": len(data.get("subjects", [])),
            "total_trials": total_trials,
        },
        "per_stream": per_stream_report,
        "missing_required_streams": {
            s: sorted(missing_required_by_stream.get(s, []))
            for s in required_streams
        },
        "per_trial_stream_counts": per_trial_stream_counts,  # useful for deeper debugging
    }

    # Pretty print a compact summary to stdout
    _print_sanity_summary(result, also_show_empty_streams=also_show_empty_streams)
    return result

def _print_sanity_summary(report: Dict, also_show_empty_streams: bool = False) -> None:
    print("=== DATASET SANITY CHECK ===")
    print(f"Total subjects: {report['summary']['total_subjects']}")
    print(f"Total trials:   {report['summary']['total_trials']}")
    print()

    print("=== PER-STREAM COVERAGE ===")
    print(f"{'stream':25} {'files':>8} {'trials_with_any':>16}")
    for s, stats in report["per_stream"].items():
        print(f"{s:25} {stats['total_files']:8d} {stats['trials_with_at_least_one']:16d}")
    print()

    # Missing required streams
    print("=== MISSING REQUIRED STREAMS ===")
    any_missing = False
    for s, trials in report["missing_required_streams"].items():
        if trials:
            any_missing = True
            print(f"- {s}: missing in {len(trials)} trial(s)")
            for t in trials:
                print(f"    • {t}")
    if not any_missing:
        print("None 🎉")
    print()

    if also_show_empty_streams:
        print("=== STREAM KEYS PRESENT BUT ZERO FILES (per stream) ===")
        for s, stats in report["per_stream"].items():
            zeros = stats.get("trials_missing_or_zero", [])
            if zeros:
                print(f"- {s}: {len(zeros)} trial(s)")
                for t in zeros:
                    print(f"    • {t}")
        print()


rep = sanity_check_dataset(json_data, required_streams=("smartwatch_data","audio"))


### Populate Key Steps using VIA Annotations

In [ ]:
import json

def add_keysteps_to_json(
    existing_json,
    keystep_json_path,
    subject_id="ng1",
    scenario_id="cardiac_arrest",
    trial_id="1",
    gopro_file_name="gopro_1.mp4"
):
    # --- load annotations & mappings ---
    with open(keystep_json_path, 'r') as f:
        keystep_data = json.load(f)
    with open('./class_id_mappings.json', 'r') as f:
        class_id_mapping = json.load(f)

    print("Adding keysteps to the JSON structure")
    print(f"Subject ID: {subject_id}, Trial ID: {trial_id}, GoPro file name: {gopro_file_name}")

    # find which vid‐key in keystep_data['file'] matches your GoPro filename
    correct_vid_id = None
    for vid_key, file_meta in keystep_data.get('file', {}).items():
        print(f"Checking file meta: {file_meta.get('fname', '')} against {gopro_file_name}")
        if gopro_file_name in file_meta.get('fname', ''):
            print("Found matching keystep file entry:", file_meta['fname'])
            correct_vid_id = vid_key
            break
        if gopro_file_name.split('_')[0] in file_meta.get('fname', ''):
            print("Found matching keystep file entry (partial match):", file_meta['fname'])
            correct_vid_id = vid_key
            break

    # build the keysteps list
    keysteps = []
    for ks_id, meta in keystep_data.get('metadata', {}).items():
        # only include those for our video, if desired:
        print(f"Video ID {meta.get('vid')} VS correct_vid_id {correct_vid_id}")
        if correct_vid_id and meta.get('vid') != correct_vid_id:
            continue
        
        print(f"Processing keystep {ks_id} for video {meta}")

        try:
            start_t, end_t = meta['z']
            label = meta['av']['1']
        except Exception as e:
            print(f"  → skipping {ks_id}: missing z or av/1")
            continue

        if label not in class_id_mapping.get('keysteps', {}):
            print(f"  → skipping {ks_id}: “{label}” not a keystep")
            continue

        if task == "cpr_quality" and label != "chest_compressions":
            print(f"  → skipping {ks_id}: “{label}” not a keystep for CPR quality task")
            continue
            
        keysteps.append({
            "keystep_id": ks_id,
            "start_t":    start_t,
            "end_t":      end_t,
            "label":      label,
            "class_id":   class_id_mapping['keysteps'][label]
        })

    # now inject into the right trial in the new hierarchy
    found = False
    for subj in existing_json.get('subjects', []):
        if subj.get('subject_id') != subject_id:
            continue
        for scen in subj.get('scenarios', []):
            if scen.get('scenario_id') != scenario_id:
                continue
            for trial in scen.get('trials', []):
                if trial.get('trial_id') == trial_id:
                    trial['keysteps'] = keysteps
                    print(f"→ Inserted {len(keysteps)} keysteps into "
                          f"subject={subject_id}, scenario={scen['scenario_id']}, trial={trial_id}")
                    found = True
    if not found:
        print(f"Warning: no matching subject={subject_id} / trial={trial_id} found in existing_json")

    return existing_json


In [ ]:

with open(output_file, 'r') as json_file:
    data = json.load(json_file)
    updated_json = data

    for subject in updated_json.get('subjects', []):
        print("*" * 50)
        # if subject['subject_id'] != 'cars_1': continue

        for scenario in subject.get('scenarios', []):
            print(f" Scenario: {scenario['scenario_id']}")
            
            for trial in scenario.get('trials', []):
                print(f"  Trial: {trial['trial_id']}")
                
                streams = trial.get('streams', {})
                # only care about GoPro streams
                if 'ego' not in streams:
                    continue

                annotation_file = None
                gopro_file_name = None

                for file in streams['ego']:
                    fp = file.get('file_path', '')

                    print(f"    File: {file['file_id']} @ {fp}")

                    # pick out the .json annotation
                    if fp.endswith('.json'):
                        annotation_file = fp
                        print(f"    Annotation: {file['file_id']} @ {fp}")

                    # pick out the matching .mp4
                    if fp.endswith('rgb_final.mp4') or fp.endswith('rgb_partial.mp4'):
                        gopro_file_name = file['file_id'] + '.mp4'
                        print(f"    GoPro video: {gopro_file_name}")

                

                if annotation_file and gopro_file_name:
                    # inject keysteps
                    updated_json = add_keysteps_to_json(
                        updated_json,
                        annotation_file,
                        subject_id=subject['subject_id'],
                        scenario_id=scenario['scenario_id'],
                        trial_id=trial['trial_id'],
                        gopro_file_name=gopro_file_name
                    )

                    # remove that .json from the stream listing
                    trial['streams']['ego'] = [
                        f for f in trial['streams']['ego']
                        if not f['file_path'].endswith('.json')
                    ]
                else:
                    print(f"  → no annotation file or GoPro video found for {subject['subject_id']} / {scenario['scenario_id']} / {trial['trial_id']}")

    # write back out
    with open(output_file, 'w') as json_file:
        json.dump(updated_json, json_file, indent=4)

    print(f"\nUpdated JSON saved to {output_file}")
    print("*" * 50)


## Convert any list to singular objects in stream

In [ ]:
with open(output_file, 'r') as json_file:
    updated_json = json.load(json_file)

    for subject in updated_json.get('subjects', []):
        print("*" * 50)
        # if subject['subject_id'] != 'ng8': continue

        for scenario in subject.get('scenarios', []):
            print(f" Scenario: {scenario['scenario_id']}")

            for trial in scenario.get('trials', []):
                print(f"  Trial: {trial['trial_id']}")

                # We need to iterate over a list of keys since we'll be modifying the dict
                for stream_type in list(trial.get('streams', {}).keys()):
                    files = trial['streams'][stream_type]
                    print(f"   {stream_type}: {files} (count={len(files) if isinstance(files, list) else 'N/A'})")

                    # If it's a list, collapse it to just the first element
                    if isinstance(files, list) and files:
                        trial['streams'][stream_type] = files[0]
                        print(f"    → flattened to: {trial['streams'][stream_type]}")

    # Write the updated JSON back out
    with open(output_file, 'w') as json_file:
        json.dump(updated_json, json_file, indent=4)

    print(f"\nUpdated JSON saved to {output_file}")
    print("*" * 50)

# Update Expertise Levels

In [ ]:
# load the json file
expertise_level_file = './subject_expertise_level.json'
# load data to a dictionary
with open(expertise_level_file) as json_file:
    expertise_level_data = json.load(json_file)['EXPERTISE_LEVELS']

print(expertise_level_data)

# iterate over the subjects and trials in main_annotation.json and update the expertise level
with open(output_file, 'r') as json_file:
    data = json.load(json_file)
    updated_json = data
    
    for subject in updated_json['subjects']:
        print("*" * 50)
        subject_id = subject['subject_id']
        expertise_level = expertise_level_data.get(subject_id, None)
        if expertise_level:
            subject['expertise_level'] = expertise_level
            print(f"Updated expertise level for subject {subject_id} to {expertise_level}")
        else:
            print(f"Expertise level not found for subject {subject_id}")

    with open(output_file, 'w') as json_file:
        json.dump(updated_json, json_file, indent=4)
    print(f"JSON structure saved to {output_file}")
    print("*" * 50)

# Remove trials that has no keysteps

In [ ]:
# remove trials that have no keysteps
if task == "cpr_quality":  
    with open(output_file, 'r') as json_file:
        data = json.load(json_file)
        updated_json = data
        
        # Initialize an empty list to hold subjects with non-empty trials
        subjects_with_trials = []
        
        for subject in updated_json['subjects']:
            print("*" * 50)
            for scenario in subject.get('scenarios', []):
            
                # Remove trials with no keysteps
                scenario['trials'] = [trial for trial in scenario['trials'] if trial['keysteps']]
                
                if scenario['trials']:
                    subjects_with_trials.append(subject)
                    for trial in scenario['trials']:
                        print(f"Trial {trial['trial_id']} has keysteps")
                else:
                    print(f"Removing subject {subject['subject_id']} as all trials are empty")
            
            # Update the JSON structure to only include subjects with non-empty trials
            updated_json['subjects'] = subjects_with_trials
            
        # Save the modified JSON structure back to the file
        with open(output_file, 'w') as json_file:
            json.dump(updated_json, json_file, indent=4)
        print(f"JSON structure saved to {output_file}")
        print("*" * 50)


# Fill gaps of annotations with no_action

In [ ]:

def fill_no_action_keysteps(trial):
    keysteps = sorted(trial.get("keysteps", []), key=lambda x: x["start_t"])
    if not keysteps:
        return
    no_action_class_id = 15
    filled = []

    for i in range(len(keysteps) - 1):
        curr = keysteps[i]
        nxt  = keysteps[i + 1]
        filled.append(curr)
        if nxt["start_t"] > curr["end_t"]:
            filled.append({
                "keystep_id": f"no_action_{i}",
                "start_t":    curr["end_t"],
                "end_t":      nxt["start_t"],
                "label":      "no_action",
                "class_id":   no_action_class_id
            })

    filled.append(keysteps[-1])
    trial["keysteps"] = filled

def process_subject_trials(subject):
    # now loop through scenarios first, then trials
    for scenario in subject.get("scenarios", []):
        for trial in scenario.get("trials", []):
            fill_no_action_keysteps(trial)
    return subject

def process_all_subjects(data):
    for subject in data.get("subjects", []):
        process_subject_trials(subject)
    return data

# if task == "segmentation":
#     # Load the JSON data
#     with open(output_file, "r") as f:
#         data = json.load(f)

#     # Process all subjects → scenarios → trials
#     updated = process_all_subjects(data)

#     # Save the updated JSON with filled "no_action" keysteps
#     with open(output_file, "w") as f:
#         json.dump(updated, f, indent=4)

#     print("Gaps filled with 'no_action' keysteps successfully.")


# Remove overlapping segments


In [ ]:

def split_keystep(last_keystep, current_keystep):
    """
    Adjusts the last keystep by splitting it so that it does not overlap with the current keystep.
    """
    if current_keystep['start_t'] < last_keystep['end_t']:
        # Trim the end of the last keystep so it ends exactly when the current one begins
        last_keystep['end_t'] = current_keystep['start_t']
    return last_keystep

def remove_and_split_overlapping_keysteps(trials):
    """
    For each trial in the list, sort its keysteps by start time,
    then trim any overlap between successive keysteps.
    """
    for trial in trials:
        keysteps = sorted(trial.get("keysteps", []), key=lambda x: x['start_t'])
        non_overlapping = []

        for current in keysteps:
            if not non_overlapping:
                non_overlapping.append(current)
            else:
                last = non_overlapping[-1]
                # If they overlap, split the last one
                if current['start_t'] < last['end_t']:
                    last = split_keystep(last, current)
                    non_overlapping[-1] = last
                non_overlapping.append(current)

        trial["keysteps"] = non_overlapping

if task == "segmentation":
    # Load the JSON file
    with open(output_file, 'r') as f:
        data = json.load(f)

    # Walk through subjects → scenarios → trials
    for subject in data.get("subjects", []):
        for scenario in subject.get("scenarios", []):
            remove_and_split_overlapping_keysteps(scenario.get("trials", []))

    # Save the updated JSON back to disk
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)

    print("Overlapping keysteps have been removed and saved to", output_file)


# Load JSON and Test

In [ ]:
import json

# Load JSON data from file
with open(output_file, 'r') as f:
    data = json.load(f)

print("Total subjects:", len(data.get('subjects', [])))

trial_count = 0
keystep_count = 0

trials_without_keysteps = []

for subject in data.get('subjects', []):
    print("*" * 50)
    subject_id = subject.get('subject_id')
    scenarios = subject.get('scenarios', [])
    print(f"Subject ID: {subject_id}")
    print(f"Total scenarios: {len(scenarios)}")
    
    for scenario in scenarios:
        scenario_id = scenario.get('scenario_id')
        trials = scenario.get('trials', [])
        print(f" Scenario ID: {scenario_id}")
        print(f"  Total trials: {len(trials)}")
        
        for trial in trials:
            trial_id = trial.get('trial_id')
            keysteps = trial.get('keysteps', [])
            print(f"   Trial ID: {trial_id}")
            print(f"    Total keysteps: {len(keysteps)}")
            trial_count += 1
            keystep_count += len(keysteps)

            if len(keysteps) == 0:
                trials_without_keysteps.append(f"{subject_id}/{scenario_id}/{trial_id}")

            # get the streams
            streams = trial.get('streams', {})
            print(f"    Total streams: {len(streams)}")
            for stream_type, files in streams.items():
                if isinstance(files, list):
                    print(f"     {stream_type}: {len(files)} files")
                else:
                    print(f"     {stream_type}: {files}")

    print("*" * 50)

print("Total trials:", trial_count)
print("Total keysteps:", keystep_count)

print("Trials without keysteps:")
for t in trials_without_keysteps:
    print(" -", t)


In [ ]:
import json
import pandas as pd

def get_keystep_distribution(subjects):
    keysteps_data = []

    # Extract keystep information from all subjects, scenarios, and trials
    for subject in subjects:
        for scenario in subject.get('scenarios', []):
            for trial in scenario.get('trials', []):
                for keystep in trial.get('keysteps', []):
                    keysteps_data.append({
                        'label':    keystep['label'],
                        'class_id': keystep['class_id']
                    })

    # Convert to DataFrame for easier analysis
    df = pd.DataFrame(keysteps_data)

    # Get the distribution of keysteps by label and by class_id
    label_distribution = df['label'].value_counts()
    class_distribution = df['class_id'].value_counts()

    return label_distribution, class_distribution

# Load the JSON file
with open(output_file, 'r') as f:
    data = json.load(f)

# Get the keystep distribution
label_distribution, class_distribution = get_keystep_distribution(data['subjects'])

# Display the results
print("Keystep Distribution by Label:")
print(label_distribution)

print("\nKeystep Distribution by Class ID:")
print(class_distribution)


# Plot keysteps

In [ ]:
import json
import matplotlib.pyplot as plt
import pandas as pd

def plot_keysteps_for_trial(trial, subject_id, scenario_id):
    keysteps_data = []

    # Extract keystep information from the trial
    for ks in trial.get('keysteps', []):
        keysteps_data.append({
            'label':    ks['label'],
            'start':    ks['start_t'],
            'end':      ks['end_t'],
            'duration': ks['end_t'] - ks['start_t']
        })

    # Convert to DataFrame for easier plotting
    df = pd.DataFrame(keysteps_data)

    plt.figure(figsize=(10, 6))
    for _, row in df.iterrows():
        plt.barh(row['label'], row['duration'], left=row['start'], height=0.4)

    # Add labels and title
    plt.xlabel('Time (seconds)')
    plt.ylabel('Keysteps')
    plt.title(f'Subject {subject_id} Scenario {scenario_id} Trial {trial["trial_id"]} Keystep Timeline')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_keysteps_for_all_trials(subjects):
    for subject in subjects:
        subject_id = subject.get('subject_id')
        for scenario in subject.get('scenarios', []):
            scenario_id = scenario.get('scenario_id')
            for trial in scenario.get('trials', []):
                plot_keysteps_for_trial(trial, subject_id, scenario_id)
        break  # only plot first subject's trials

# with open(output_file, 'r') as f:
#     data = json.load(f)
# plot_keysteps_for_all_trials(data.get('subjects', []))


# Generating Production Ready Version

In [ ]:
import json



def strip_base_path(data, base_dir):
    """
    Recursively remove the leading base_dir from all file_path entries.
    e.g. '/standard/UVA-DSA/NIST EMS Project Data/EgoEMS_AAAI2026/P0/...' ->
         'P0/...'
    """
    base_dir = os.path.abspath(base_dir)

    def _strip_path(path):
        try:
            abs_path = os.path.abspath(path)
            if abs_path.startswith(base_dir):
                rel = os.path.relpath(abs_path, base_dir)
                return rel.replace("\\", "/")
            return path
        except Exception:
            return path

    for subj in data.get("subjects", []):
        for scen in subj.get("scenarios", []):
            for trial in scen.get("trials", []):
                streams = trial.get("streams", {})
                for _name, item in streams.items():
                    if isinstance(item, dict) and "file_path" in item:
                        item["file_path"] = _strip_path(item["file_path"])
    return data

if is_production:
    print("Stripping base paths for production deployment...")
    # Load JSON data from file
    with open(output_file, 'r') as f:
        data = json.load(f)

    print("Total subjects:", len(data.get('subjects', [])))

    data = strip_base_path(data, "/standard/UVA-DSA/NIST EMS Project Data/EgoEMS_AAAI2026")

    # Save the updated JSON back to file
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)
    print(f"Updated JSON with stripped paths saved to {output_file}")